In [1]:
import z3
from pyspec import *

In [2]:
A = z3.DeclareSort('A')
print(type(A), str(A))

<class 'z3.z3.SortRef'> A


In [3]:
A = make_z3sort_expression(z3.DeclareSort('A'))
B = make_z3sort_expression(z3.DeclareSort('B'))

x,y,z = A.mk_atoms('x,y,z')

r,s,t = B.mk_atoms('r,s,t')

def eval_expr(es):
    print(es+': ', str(eval(es)))

examples = [(
    f'{x} + {y} + {z}',
    f'{x} + ({y+z})',
    f'{x} < {x} << {z}',
    f'{x} > {x} << {z}',
    f'{x} << {y} << {z}',
    f'{x}-{y} << {z}',
    f'{y} < {z}',
    f'{x}+{y}+{x}+{y}'
    )
    for (x,y,z) in [(x,y,z),(r,s,t)]
]

for ts in examples:
    for e in ts:
        eval_expr(e)


x + y + z:  x+y+z
x + (y+z):  x+(y+z)
x < x << z:  x<(x<<z)
x > x << z:  x>(x<<z)
x << y << z:  x<<y<<z
x-y << z:  x-y<<z
y < z:  y<z
x+y+x+y:  x+y+x+y
r + s + t:  r+s+t
r + (s+t):  r+(s+t)
r < r << t:  r<(r<<t)
r > r << t:  r>(r<<t)
r << s << t:  r<<s<<t
r-s << t:  r-s<<t
s < t:  s<t
r+s+r+s:  r+s+r+s


In [4]:
try:
    x*y // z + y ^ t+r
except TypeError as te:
    print("Exception:", str(te))

Exception: Operands of ^ operator have incompatible types: 'x*y//z+y: A_Add' versus 't+r: B_Add'


In [5]:
print(A.__name__)

A_Z3SortExpression


In [6]:
add_x_y = x+y
print(add_x_y)
print(add_x_y.z3Func)
#type(add_x_y.z3Expr)


x+y
z3_Add
